## Exercize 1.1
### Word Similarity
#### Francesco Sannicola

Import wordnet interface from nltk

In [1]:
from nltk.corpus import wordnet as wn

Read the file which contains couples of word with a human made similarity score

In [2]:
import csv

def read_csv():
    all_col = []
    with open('./Input/WordSim353.csv', 'r', encoding='utf-8-sig') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=';')
        for col in csv_reader:
            all_col.append(col)
    return all_col

Split the input file readed into:
- base: first word
- target: second word
- gold: similitary score

In [3]:
input_file_similarity = read_csv()
input_file_similarity.pop(0)

base = []
target = []
gold = []

for row in input_file_similarity:
    field = row[0].split(',')
    base.append(field[0])
    target.append(field[1])
    gold.append(float(field[2]))

Method that calculates the lowest common subsumer. Useful for all similarity algoritms.

In [4]:
def lowest_common_subsumer(syn1, syn2):
    """
    :param syn1: first word's synset
    :param syn2: second word's synset
    :return: common lowest ancestor's synset
    """
    if syn1 == syn2:
        return syn1
    
    commons = []
    
    for s1 in syn1.hypernym_paths():
        for s2 in syn2.hypernym_paths():
            # couples of elements in the same position
            union = list(zip(s1, s2))
            common = None
            
            # find a tuple with equal components
            for i in range(len(union)):
                if union[i][0] != union[i][1]:
                    break
                common = (union[i][0], i)
            if common is not None and common not in commons:
                commons.append(common)

    if len(commons) == 0:
        return None
    
    # return and find the lowest ancestor by reversing commons and take the first one
    commons.sort(key=lambda x: x[1], reverse=True)
    return commons[0][0]

Get distance between 2 synset.

In [5]:
def get_distance(syn1, syn2):
    '''
    :param syn1: 1st synset
    :param syn2: 2nd synset
    :return: distance between the given synsets
    '''

    # find the lowest common subsumer betweeen two synsets
    lcs = lowest_common_subsumer(syn1, syn2)
    
    if lcs is None:
        return None
    
    # obtain hypernym paths for both 
    hyper_synset1 = syn1.hypernym_paths()
    hyper_synset2 = syn2.hypernym_paths()

    if syn1 == syn2:
        return 0
    
    # convert a list into a set (for remove method)
    lists_lcs_hyper = lcs.hypernym_paths()
    set_lcs_hyper = set()
    for el in lists_lcs_hyper:
        for i in el:
            set_lcs_hyper.add(i)
            
    # nodes from LCS to root. Delete the LCS
    set_lcs_hyper.remove(lcs)

    # path from both synsets to LCS. Map is defined as map(function, iterable, ...), 
    # function perform some action to each element of an iterable
    hyper_synset1 = list(map(lambda x: [y for y in x if y not in set_lcs_hyper], hyper_synset1))
    hyper_synset2 = list(map(lambda x: [y for y in x if y not in set_lcs_hyper], hyper_synset2))
    
    # mantain only paths with LCS. Filter do that (filter(function, iterable)) 
    hyper_synset1 = list(filter(lambda x: lcs in x, hyper_synset1))
    hyper_synset2 = list(filter(lambda x: lcs in x, hyper_synset2))
    
    # take the min path from both synsets LCS
    distance = min(list(map(lambda x: len(x), hyper_synset1))) + min(list(map(lambda x: len(x), hyper_synset2))) - 2

    return distance

Obtain the depth of a synset synset. I.e. it mesures the distance between the given synset and the WordNet's root. Result path contains LCS of 2 synset. Method useful for Wu and Palmer metric.

In [6]:
def get_depth_lcs(syn, lcs):
    '''
    :param syn: synset to reach from the root
    :param lcs: first common sense
    :return: minimum path which contains LCS
    '''
    paths = syn.hypernym_paths()
    paths = list(filter(lambda x: lcs in x, paths))
    return min(len(path) for path in paths)

First similarity algoritm:
### Wu & Palmer
$$
  sim(s_1,s_2) = \frac{2 * depth(LCS)}{depth(s_1) + depth(s_2)}
$$
 The similarity is based on WordNet structure.
 \
 LCS is the lowest common subsumer between two synset and depth(x) is the distance from root to x.
 \
 The result value is normalized to match the gold field.

In [7]:
def wup_sim(syn1, syn2):

    lcs = lowest_common_subsumer(syn1, syn2)
    if lcs is None:
        return 0
    sim = 2 * (get_depth_lcs(lcs, lcs) / (get_depth_lcs(syn1, lcs) + get_depth_lcs(syn2, lcs)))

    return sim * 10

Calculate the maximum depth of the taxonomy

In [8]:
max_depth = max(max(len(hyp_path) for hyp_path in ss.hypernym_paths()) for ss in wn.all_synsets())

### Leakcock-Chodorow:
$$
  sim(s_1,s_2) = -log \frac{len (s_1, s_2)}{2 * depthMax}
$$

When s1 and s2 have the same sense, len(s1 ,s2)=0.
\
In practice, we add 1 to both len(s1 ,s2) and 2*depthMax to avoid log(0). The result is in [0,log(2 * depthMax +1)]


In [9]:
import math

def leakcock_chodorow_sim(syn1, syn2):

    len = get_distance(syn1, syn2)
    
    if len is None:
        return 0

    if len == 0:
        sim = math.log10((len + 1) / (2 * max_depth + 1))
    else:
        sim = math.log10(len / (2 * max_depth))

    sim = sim * (-1)
    return (sim / (math.log10(2 * max_depth + 1))) * 10

Implementation of the last similarity method: 
### Shortest path.
$$
  sim(s_1,s_2) = 2 * depthMax - len(s_1, s_2)
$$
We have 3 options:
- if len(s1 ,s2) is 0, sim path (s1 ,s2) gets the maximum value of 2* depthMax.
- if len(s1 ,s2) is 2* depthMax, sim path (s1 ,s2) gets the minimum value of 0.
- thus, the values of sim path (s1 ,s2) are between 0 and 2* depthMax.


The result is normalized from [0, 2*max_depth] to [0,10]

In [10]:
def shortest_path_sim(syn1, syn2):
    len = get_distance(syn1, syn2)
    if len is None:
        return 0

    if len == 0:
        sim = 2 * max_depth

    if len == 2 * max_depth:
        sim = 0

    sim = 2 * max_depth - len
    return (sim / 40) * 10

Call all the methods implemented previously.

For a pair of polysemic words we consider the maximum probability for each sense of each word:
$$
  sim(w_1,w_2)=\max_{c1 \in s(w_1), c2 \in s(w_2)} [sim(c_1, c_2)]
$$


In [11]:
wup_val = []
lc_val = []
sp_val = []

for i in range(0, len(gold)):

    max_sim = 0
    wup = 0
    for syn1 in wn.synsets(base[i]):
        for syn2 in wn.synsets(target[i]):
            sim = wup_sim(syn1, syn2)
            if sim > max_sim:
                wup = sim
                max_sim = sim
    wup_val.append(wup)

    max_sim = 0
    lc = 0
    for syn1 in wn.synsets(base[i]):
        for syn2 in wn.synsets(target[i]):
            sim = leakcock_chodorow_sim(syn1, syn2)
            if sim > max_sim:
                lc = sim
                max_sim = sim
    lc_val.append(lc)

    max_sim = 0
    sp = 0
    for syn1 in wn.synsets(base[i]):
        for syn2 in wn.synsets(target[i]):
            sim = shortest_path_sim(syn1, syn2)
            if sim > max_sim:
                sp = sim
                max_sim = sim
    sp_val.append(sp)

Print a table with all similarity.

In [12]:
from prettytable import PrettyTable

table_res = PrettyTable(["Base", "Target", "Wu & Palmer", "Leakcock & Chodorow", "Shortest-Path", "Gold"])

for i in range(0, len(gold)):
    # print(base[i], target[i], round(wup_val[i], 2), round(lc_val[i], 2), round(sp_val[i], 2), round(gold[i], 2))
    table_res.add_row(
        [base[i], target[i], round(wup_val[i], 2), round(lc_val[i], 2), round(sp_val[i], 2), round(gold[i], 2)])

print(table_res)

+---------------+----------------+-------------+---------------------+---------------+------+
|      Base     |     Target     | Wu & Palmer | Leakcock & Chodorow | Shortest-Path | Gold |
+---------------+----------------+-------------+---------------------+---------------+------+
|      love     |      sex       |     9.23    |         9.93        |      9.75     | 6.77 |
|     tiger     |      cat       |     9.66    |         9.93        |      9.75     | 7.35 |
|     tiger     |     tiger      |     10.0    |         10.0        |      10.0     | 10.0 |
|      book     |     paper      |     8.75    |         8.07        |      9.5      | 7.46 |
|    computer   |    keyboard    |     8.24    |         6.98        |      9.25     | 7.62 |
|    computer   |    internet    |     6.32    |         4.69        |      8.25     | 7.58 |
|     plane     |      car       |     7.27    |         5.11        |      8.5      | 5.77 |
|     train     |      car       |     7.37    |         5.6

Import some useful library to compute correlation coeffincents.

In [13]:
import numpy
from numpy import cov, std
from scipy.stats import rankdata

Implementation of Spearman's rank correlation coefficent:
$$
  r_s= \frac{cov (rg_x, rg_y)}{σ_{rg_{X}} * σ_{rg_{Y}} }
$$

In [14]:
def spearman_index(target, predicted):
    '''
    :param target: gold value
    :param predicted: computed value
    :return: value of the spearman's correlation
    '''
    target = numpy.array(target).astype(numpy.float)
    predicted = numpy.array(predicted).astype(numpy.float)
    return cov(rankdata(target), rankdata(predicted))[0][1] / (std(rankdata(target)) * std(rankdata(predicted)))

Implementation of Pearson's correlation coefficent:
$$
  r_s= \frac{cov (X, Y)}{σ_{X} * σ_{Y} }
$$

In [15]:
def pearson_index(target, predicted):
    '''
        :param target: gold value
        :param predicted: computed value
        :return: value of the pearson's correlation
    '''
    target = numpy.array(target).astype(numpy.float)
    predicted = numpy.array(predicted).astype(numpy.float)
    return cov(target, predicted)[0][1] / (std(target) * std(predicted))

Print correlation result of both technique for all similarity metrics.

In [16]:
table_res_pearson = PrettyTable(["Coefficent", "Value"])

print ("-----------------Pearson coefficent-----------------")
table_res_pearson.add_row(["Wu & Palmer", pearson_index(gold, wup_val)])
table_res_pearson.add_row(["Leakcock & Chodorow", pearson_index(gold, lc_val)])
table_res_pearson.add_row(["Shortest-Path", pearson_index(gold, sp_val)])
print(table_res_pearson)

table_res_spearman = PrettyTable(["Coefficent", "Value"])
print ("-----------------Spearman coefficent-----------------")
table_res_spearman.add_row(["Wu & Palmer", spearman_index(gold, wup_val)])
table_res_spearman.add_row(["Leakcock & Chodorow", spearman_index(gold, lc_val)])
table_res_spearman.add_row(["Shortest-Path", spearman_index(gold, sp_val)])
print(table_res_spearman)

-----------------Pearson coefficent-----------------
+---------------------+---------------------+
|      Coefficent     |        Value        |
+---------------------+---------------------+
|     Wu & Palmer     |  0.2881726141884054 |
| Leakcock & Chodorow | 0.31827728260366556 |
|    Shortest-Path    | 0.16458563735418705 |
+---------------------+---------------------+
-----------------Spearman coefficent-----------------
+---------------------+---------------------+
|      Coefficent     |        Value        |
+---------------------+---------------------+
|     Wu & Palmer     | 0.33724305240662467 |
| Leakcock & Chodorow | 0.28912011037697033 |
|    Shortest-Path    | 0.28912011037697033 |
+---------------------+---------------------+


##### The result shows that all three similarity metric does not perform very well. This is predictable considering task difficulty.

##### However, seems that the Wu & Palmer and the Leakcock & Chodorow have better performance compared to the Shortest-Path metric.